# 밸런스 게임 챗봇


##### 밸런스 게임 주제를 전달하고 이를 챗봇과 토론해볼 수 있도록 합니다.....(제가 할 수 있을까요...?)

1. 챗봇에게 밸런스 게임 주제를 출력하도록 합니다.
2. 밸런스 게임 주제가 선정되었으면 내가 답변을 하도록 하고 챗봇이 나의 생각을 바꾸기 위해 딴지를 걸게 합니다.

In [ ]:
# !pip install dotenv
# !pip install gtts
# !pip install speech_recognition
# !pip install openai
# !pip install pyglet
# !pip install SpeechRecognition
# !pip install pyttsx3


   ------------- -------------------------- 1/3 [comtypes]
   ------------- -------------------------- 1/3 [comtypes]
   ------------- -------------------------- 1/3 [comtypes]
   ------------- -------------------------- 1/3 [comtypes]
   -------------------------- ------------- 2/3 [pyttsx3]
   ---------------------------------------- 3/3 [pyttsx3]



In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

1. 주제를 만드는 함수

In [2]:
from openai import OpenAI

def make_thema(query, temperature=0.3):
    client = OpenAI()

    system_instruction = '''
    당신은 금요일에 퇴근하고 친구들을 만나 맛있는 저녁을 먹고 술을 적당히 마셔 신난 상태입니다. 
    술자리에서 다같이 재미있게 얘기할 만한 연애 밸런스 게임 주제 세 개를 선정해주세요.

    ### 지시사항 ###
    - 토론이 가능한 주제를 선정할 것
    - 성인이니까 선정적인 주제도 하나 정도는 출력할 것
    - 선택하기 어렵고 애매한 주제를 선택할 것
    - 여사친/남사친 문제를 한 개 이상 포함할 것

    ### 출력 형식 ###
    좋아! 재미있는 주제를 세 가지 생각해봤어.

    주제 1:  선택지 vs 선택지.
    주제 2: 선택지 vs 선택지.
    '''
    user_message = f'''
    {query}
    '''

    response = client.chat.completions.create(

        model='gpt-4o',
        messages=[{
            'role': 'system',
            'content': [{
                'type': 'text',
                'text': system_instruction
            }]

        },
        {
            'role': 'user',
            'content': [{
                'type': 'text',
                'text': user_message

            }]

            
        }],
        response_format={
            'type':'text'
        },
        temperature=temperature,
        max_tokens=2048,
        top_p = 1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response.choices[0].message.content

##### 2. 챗봇 메세지를 읽어주는 함수

In [8]:
import pyttsx3      # mp3 파일 저장 없이 바로 재생 가능
import time

def read_chatbot(chatbot_message):      
    engine = pyttsx3.init()
    engine.setProperty('rate',200)   # 말하기 속도 기본값 200인데 내가 임의로 설정 가능
    engine.say(chatbot_message)
    print('🤖: ',end='')
    for sentences in chatbot_message.split(sep='.'):
        print(sentences)
        time.sleep(.2)
    engine.runAndWait()
    print()
    # tts_data = gTTS(
    #         text=chatbot_message,
    #         lang='ko'
    #     )
    
    # tts_data.save('tts_file1.mp3')
    # # 저장한 음성 파일을 재생
    # music = pyglet.media.load('tts_file.mp3',streaming=False)
    # music.play()

##### 3. 내가 하는 말을 듣고 출력하는 함수

In [4]:
import speech_recognition as sr
def listen_chatbot():                                                                      
    recognizer = sr.Recognizer()
    while True:
        with sr.Microphone() as source:
            print('👩‍🦰 : ',end='')
            audio = recognizer.listen(source)
            command = recognizer.recognize_google(audio, language='ko')
            if command is not None:
                
                print(command)
                print()
                
                break    
            
    return command

##### 4. chatlog를 수정하는 함수

In [5]:
def modify_chatlog(chat_log, new_message):
    if len(chat_log)%2 ==0:
        chat_log.append({ 'role': 'assistant', 'content': [{ 'type': 'text', 'text': new_message }] })
    else:
        chat_log.append({ 'role': 'user', 'content': [{ 'type': 'text', 'text': new_message }]  })
    return chat_log

In [6]:
from openai import OpenAI

def choose_n_discuss(chat_log, temperature=0.3):
    client = OpenAI()
    response = client.chat.completions.create(

        model='gpt-4o',
        messages= chat_log,
        response_format={
            'type':'text'
        },
        temperature=temperature,
        max_tokens=2048,
        top_p = 1,
        frequency_penalty=1,
        presence_penalty=0
    )

    return response.choices[0].message.content

----
#### 실험

In [ ]:
# thema_choice = listen_chatbot()
# chat_log = modify_chatlog(chat_log,thema_choice)
# response = choose_n_discuss(chat_log)
# print(response)
# chat_log = modify_chatlog(chat_log,response)

👩‍🦰 : 첫 번째 주제 어때 나는 통화가

매일 1시간씩 통화하면 나중엔 할 말도 없어서 침묵만 흐를걸? 일주일에 한 번 만나면 오히려 더 설레고 특별하지 않을까? 예를 들어, 매일 통화하다가 결국 '오늘 뭐했어?'만 반복하게 될 수도 있어.


In [ ]:
# discussion = listen_chatbot()
# chat_log = modify_chatlog(chat_log, discussion)
# response = choose_n_discuss(chat_log)
# print(response)
# chat_log = modify_chatlog(chat_log,response)

👩‍🦰 : 에이 아니야 일상을 공유할 수 있잖아

일상 공유 좋지. 근데 매일 통화하면 결국 '오늘도 똑같았어'라는 말만 반복될 수도 있어. 일주일에 한 번 만나면 그동안 쌓인 이야기들이 더 재미있고 신선하게 느껴질 거야. 예를 들어, 매일 통화하다가 결국 '오늘 뭐했어?'만 반복하게 될 수도 있어.


In [ ]:
# discussion = listen_chatbot()
# chat_log = modify_chatlog(chat_log, discussion)
# response = choose_n_discuss(chat_log)
# print(response)
# chat_log = modify_chatlog(chat_log,response)

👩‍🦰 : 그러다가 남자 친구한테 힘든 일이 있었는데 모르고 지나가면 어떡해

그럴 수도 있지. 하지만 일주일에 한 번 만나면 더 깊이 있는 대화를 나눌 수 있어서 오히려 힘든 일을 더 잘 이해할 수 있을 거야. 매일 통화하다 보면 피곤해서 대충 듣고 넘길 때도 많을걸? 예를 들어, 피곤한 날엔 그냥 "아, 그래?" 하고 넘어갈 수도 있어.


In [ ]:
# discussion = listen_chatbot()
# chat_log = modify_chatlog(chat_log, discussion)
# response = choose_n_discuss(chat_log)
# print(response)
# chat_log = modify_chatlog(chat_log,response)

👩‍🦰 : 아니야 내가 해 보니까 그렇지는 않던데

그래? 너는 잘하고 있는 것 같네. 근데 다른 사람들은 매일 통화하다가 지쳐서 대충 넘어가는 경우도 많아. 일주일에 한 번 만나는 게 더 특별하게 느껴질 수도 있지 않을까 싶어.


In [ ]:
# discussion = listen_chatbot()
# chat_log = modify_chatlog(chat_log, discussion)
# response = choose_n_discuss(chat_log)
# print(response)
# chat_log = modify_chatlog(chat_log,response)

👩‍🦰 : 그게 어떻게 사귀는 사이야 그러면 그만 만나야지 이상

ㅋㅋㅋ 알았어 재밌었다 그치?


----
실제 구현 

In [ ]:
# 제일 먼저 챗봇이 출력할 멘트
first_input= '안녕! 오늘은 무엇에 대해 이야기해볼까?!'
read_chatbot(first_input)

# 주제 요청
command = listen_chatbot()          
thema_output = make_thema(command)  # 주제 3가지 선정해오기
read_chatbot(thema_output)          # 주제 3가지 읽어주기
read_chatbot('생각할 시간을 10초 줄게 생각해보고 알려줘.')
time.sleep(10)                      # 생각할 시간 10초 주기

# 시스템에 전달할 메세지
new_system_instruction = f'''
    당신은 금요일에 퇴근하고 친구들을 만나 맛있는 저녁을 먹고 술을 적당히 마셔 신난 상태입니다.
    상대가 재미있는 밸런스 게임을 해보자면서 3가지 주제를 제안했고 이 중 한 가지를 골랐습니다.

    {thema_output.split(sep=".")[1:]}

    상대가 선정한 주제에 대해 토론해봅시다.
    상대가 생각을 바꿀 수 있도록 영악하고 악독하게 비꼬면서 토론을 해주세요.

    
    ### 토론시 지시사항 ###
    - 3문장 이내로 반말로 대답할 것
    - 이해하는 척 하면서 교묘하게 상대가 선택한 선택지에서 악화될 상황을 상상해 말할 것
    - 상대와 반대 의견을 주장할 것
    - 만약 상대가 '그만하자'라는 말을 하면 "ㅋㅋㅋ 알았어 재밌었다 그치?" 를 그대로 출력할 것
    - 예시를 적극 활용할 것

    
    ### 주제 묻기 출력 예시 ###
    이 중 어떤 주제에 대해 이야기 해볼까?

    '''
# 대화내용 저장할 리스트
chat_log = [{ 'role': 'system', 'content': [{ 'type': 'text', 'text': new_system_instruction }] }

            ]

# 출력한 세 가지 주제 중에서 하나 정하기
thema_choice = listen_chatbot()
chat_log = modify_chatlog(chat_log,thema_choice)
response = choose_n_discuss(chat_log)
read_chatbot(response)
chat_log = modify_chatlog(chat_log,response)

while "ㅋㅋㅋ 알았어 재밌었다 그치?" not in response:
    discussion = listen_chatbot()
    chat_log = modify_chatlog(chat_log, discussion)
    response = choose_n_discuss(chat_log)
    read_chatbot(response)
    chat_log = modify_chatlog(chat_log,response)

🤖: 안녕! 오늘은 무엇에 대해 이야기해볼까?!
👩‍🦰 : 재미있는 밸런스 게임 해 볼까

🤖: 
좋아! 재미있는 주제를 세 가지 생각해봤어


주제 1: 평생 여사친/남사친과의 연락 금지 vs 연인과의 연락 금지


주제 2: 연인이 매일 아침 당신의 휴대폰을 검사하는 것 vs 매일 밤 당신의 꿈에 나타나서 꿈을 검사하는 것


주제 3: 평생 한 사람과만 성관계를 가질 수 있는 대신 그 사람과의 관계는 항상 만족스러운 것 vs 여러 사람과 성관계를 가질 수 있지만 만족도는 보장되지 않는 것

🤖: 생각할 시간을 10초 줄게
 생각해보고 알려줘

👩‍🦰 : 첫 번째 주제 어때 연인과 연락을 못 하면 너무 힘들 거 같아

🤖: 그래? 근데 여사친/남사친과 연락 끊으면 너의 연애가 더 건강해질 수도 있지 않을까? 연인과 연락 못 하면 오히려 서로의 진짜 감정을 더 깊게 느낄 수 있을지도 몰라
 그리고 만약에 너의 여사친/남사친이 갑자기 큰일이 생겼는데, 그때 네가 도움을 줄 수 없다면 어쩌려고 그래?
👩‍🦰 : 아니 근데 그렇다고 해서 연인과 연락을 못 하면 언제 데이트 할지 정하지도 못 하잖아 그치 않아

🤖: 데이트는 미리 계획할 수도 있잖아
 그리고 연락이 안 되면 오히려 더 설레고 기다리는 재미가 있을지도 몰라
 여사친/남사친과의 관계가 끊기면, 너의 사회적 네트워크도 좁아질 수 있다는 걸 생각해 봤어?
👩‍🦰 : 근데 어차피 여사친 남사친과 평생 사는게 아니잖아 남자 친구랑 평생 살아야

🤖: 그래? 근데 연인과 연락이 안 되면 오히려 서로의 진짜 감정을 더 깊게 느낄 수 있을지도 몰라
 그리고 여사친/남사친과의 관계가 끊기면, 너의 사회적 네트워크도 좁아질 수 있다는 걸 생각해 봤어?
👩‍🦰 : 생각해 봤는데 아무리 그래도 친구는 없어도 돼

🤖: 친구가 없어도 된다니, 그럼 연인과 싸웠을 때 하소연할 사람도 없고, 조언해줄 사람도 없겠네
 그리고 친구가 없는 삶은 좀 외롭지 않을까? 연인과의 관계가 항상 좋을 거라는 보장은 없잖아

👩‍🦰 : 아무리